In [4]:
# Use Google Colab
use_colab = True
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive._mount('/content/drive')
    %cd '/content/drive/My Drive/685'
    !pip install -r requirements.txt

!pip install sentencepiece

from datasets import load_dataset
import sys
import os
import random
import shutil
import copy
import inspect

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm
#from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/685
     |████████████████████████████████| 12.0 MB 3.8 MB/s 
     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 298 kB 41.4 MB/s 
     |████████████████████████████████| 265 kB 45.9 MB/s 
     |████████████████████████████████| 10.4 MB 15.3 MB/s 
     |████████████████████████████████| 1.0 MB 20.7 MB/s 
     |████████████████████████████████| 61 kB 480 kB/s 
     |████████████████████████████████| 3.3 MB 40.2 MB/s 
     |████████████████████████████████| 596 kB 30.7 MB/s 
     |████████████████████████████████| 895 kB 33.3 MB/s 
     |████████████████████████████████| 243 kB 48.7 MB/s 
     |████████████████████████████████| 1.1 MB 37.5 MB/s 
     |████████████████████████████████| 132 kB 48.9 MB/s 
     |████████████████████████████████| 192 kB 51.1 MB/s 
     |███████████████████████████

In [5]:
assert torch.cuda.is_available()
dataset = load_dataset('csv', data_files={'train': 'hateval2019_en_train.csv','test': 'hateval2019_en_test.csv','valid': 'hateval2019_en_dev.csv'})
# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Using custom data configuration default-24d0d3aaf0edc963


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-24d0d3aaf0edc963/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Found device: Tesla K80, n_gpu: 1


In [6]:
def mappingFunction(index, Data):
# NOT            0 0 0 -> 0
# HSP NOT  NOT   1 0 0 -> 1
# HSP TRG  NOT   1 1 0 -> 2
# HSP NOT  ARG   1 0 1 -> 3
# HSP TRG  ARG   1 1 1 -> 4

  if Data["HS"][index] == 0 and Data["TR"][index]== 0 and Data["AG"][index] == 0:
    return 0
  elif Data["HS"][index] == 1 and Data["TR"][index]== 0 and Data["AG"][index] == 0:
    return 1
  elif Data["HS"][index] == 1 and Data["TR"][index]== 1 and Data["AG"][index] == 0:
    return 2
  elif Data["HS"][index] == 1 and Data["TR"][index]== 0 and Data["AG"][index] == 1:
    return 3
  elif Data["HS"][index] == 1 and Data["TR"][index]== 1 and Data["AG"][index] == 1:
    return 4

In [ ]:
# dataset["train"]

In [ ]:
# dataset["train"][0]

In [7]:
# Convert to Multi-Class
classCol = [0] * len(dataset["train"])

for index in range(len(dataset["train"])):
  classCol[index]  =  mappingFunction(index, dataset["train"])
dataset["train"] = dataset["train"].add_column("class", classCol)

In [8]:
classCol = [0] * len(dataset["valid"])

for index in range(len(dataset["valid"])):
  classCol[index]  =  mappingFunction(index, dataset["valid"])
dataset["valid"] = dataset["valid"].add_column("class", classCol)

In [9]:
classCol = [0] * len(dataset["test"])

for index in range(len(dataset["test"])):
  classCol[index]  =  mappingFunction(index, dataset["test"])
dataset["test"] = dataset["test"].add_column("class", classCol)

In [10]:
dataset["train"]

Dataset({
    features: ['id', 'text', 'HS', 'TR', 'AG', 'class'],
    num_rows: 9000
})

In [ ]:
dataset["test"]

Dataset({
    features: ['id', 'text', 'HS', 'TR', 'AG', 'class'],
    num_rows: 3000
})

In [ ]:
dataset["valid"]

Dataset({
    features: ['id', 'text', 'HS', 'TR', 'AG', 'class'],
    num_rows: 1000
})

In [11]:
train_dataloader = torch.utils.data.DataLoader(dataset['train'], shuffle=True, batch_size=8)
val_dataloader = torch.utils.data.DataLoader(dataset['valid'], shuffle=True, batch_size=8)
test_dataloader = torch.utils.data.DataLoader(dataset['test'], shuffle=True, batch_size=8)

In [13]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 5, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    cache_dir='./bert_cache'
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tell pytorch to run this model on the GPU.
model.cuda()

batch_size = 99
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 5

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# os.mkdir("trained_model_hateval_baseline_multiclass")

In [14]:
import numpy as np
from sklearn.metrics import classification_report
# function to get validation accuracy
def get_validation_performance(val_dataloader):
    # Put the model in evaluation mode
    model.eval()
    y_pred = []
    y_true = []
    # Tracking variables 
    total_eval_accuracy = 0
    #total_eval_loss = 0

    total_correct = 0

    for batch in val_dataloader:
      if len(batch) == 0: continue
      inputs = tokenizer(batch['text'],padding='longest',return_tensors="pt").to(device)
      labels = batch['class'].to(device)
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits
            
        # Accumulate the validation loss.
        #total_eval_loss += loss.item()
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        y_true.extend(labels_flat)
        y_pred.extend(pred_flat)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / (len(val_dataloader) * val_dataloader.batch_size)
    report = classification_report(y_true=y_true, y_pred=y_pred)
    print ("val accuracy: ", report)
    return avg_val_accuracy


In [16]:
import random

for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    total_train_loss = 0
    model.train()
    for batch in tqdm.notebook.tqdm(train_dataloader):
      if len(batch) == 0: continue
      inputs = tokenizer(batch['text'],padding='longest',return_tensors="pt").to(device)
      labels = batch['class'].to(device)
      # Clear the previously calculated gradient
      model.zero_grad()        
      # Perform a forward pass (evaluate the model on this training batch).
      outputs = model(**inputs, labels=labels)
      loss = outputs.loss
      logits = outputs.logits
      total_train_loss += loss.item()
      # Perform a backward pass to calculate the gradients.
      loss.backward()
      # Update parameters and take a step using the computed gradient.
      optimizer.step()
    # ========================================
    #               Validation
    # ========================================
    print(f"Total loss: {total_train_loss}")
    val_acc = get_validation_performance(val_dataloader)
    print(f"Validation accuracy: {val_acc}")
    torch.save(model.state_dict(),os.path.join('./trained_model_hateval_multiclass/', 'baseline_{}.pt'.format(epoch_i)))
    
print("")
print("Training complete!")

#get_validation_performance(test_dataloader)


======== Epoch 1 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 967.1191003397107
val accuracy:                precision    recall  f1-score   support

           0       0.74      0.89      0.81       573
           1       0.24      0.04      0.07       113
           2       0.57      0.56      0.57       110
           3       0.54      0.65      0.59        95
           4       0.64      0.39      0.49       109

    accuracy                           0.68      1000
   macro avg       0.55      0.51      0.51      1000
weighted avg       0.64      0.68      0.64      1000

Validation accuracy: 0.683

======== Epoch 2 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 718.9950112961233
val accuracy:                precision    recall  f1-score   support

           0       0.79      0.86      0.82       573
           1       0.24      0.05      0.09       113
           2       0.59      0.45      0.51       110
           3       0.44      0.77      0.56        95
           4       0.58      0.56      0.57       109

    accuracy                           0.68      1000
   macro avg       0.53      0.54      0.51      1000
weighted avg       0.65      0.68      0.65      1000

Validation accuracy: 0.68

======== Epoch 3 / 5 ========
Training...


...

Total loss: 507.32942962273955
val accuracy:                precision    recall  f1-score   support

           0       0.81      0.81      0.81       573
           1       0.30      0.22      0.26       113
           2       0.60      0.56      0.58       110
           3       0.50      0.57      0.53        95
           4       0.52      0.61      0.56       109

    accuracy                           0.67      1000
   macro avg       0.54      0.56      0.55      1000
weighted avg       0.67      0.67      0.67      1000

Validation accuracy: 0.672

======== Epoch 4 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 347.6899530016817
val accuracy:                precision    recall  f1-score   support

           0       0.79      0.83      0.81       573
           1       0.36      0.28      0.32       113
           2       0.48      0.58      0.53       110
           3       0.52      0.53      0.52        95
           4       0.57      0.42      0.48       109

    accuracy                           0.67      1000
   macro avg       0.54      0.53      0.53      1000
weighted avg       0.66      0.67      0.66      1000

Validation accuracy: 0.665

======== Epoch 5 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 258.64395343721844
val accuracy:                precision    recall  f1-score   support

           0       0.84      0.70      0.77       573
           1       0.26      0.41      0.32       113
           2       0.45      0.58      0.51       110
           3       0.39      0.44      0.41        95
           4       0.56      0.50      0.53       109

    accuracy                           0.61      1000
   macro avg       0.50      0.53      0.51      1000
weighted avg       0.66      0.61      0.63      1000

Validation accuracy: 0.608

Training complete!


In [17]:
#model.load_state_dict(torch.load(os.path.join('./trained_model_hateval_baseline_subtask_b/', 'subtask_b_baseline_4.pt')))

get_validation_performance(test_dataloader)

val accuracy:                precision    recall  f1-score   support

           0       0.87      0.20      0.33      1740
           1       0.17      0.46      0.25       320
           2       0.33      0.67      0.44       346
           3       0.28      0.63      0.39       411
           4       0.40      0.22      0.29       183

    accuracy                           0.34      3000
   macro avg       0.41      0.44      0.34      3000
weighted avg       0.62      0.34      0.34      3000



0.3443333333333333